In [34]:
from gensim import downloader
import pandas as pd
import random

In [74]:
def get_details(dataframe,wv_model):

    details_list = []
    #going through dataset and evaluating the closest synonym to the question
    for index, row in dataframe.iterrows():
        question=row['question']
        answer=row['answer']
        system_guess_word=''
        best_choice_similarity=-1
        
        if question in wv_model.key_to_index:        
            for index in range(0,4):
                choice=row[str(index)]
                if choice in wv_model.key_to_index:
                    choice_similarity=wv_model.similarity(question,choice)
                    if choice_similarity>best_choice_similarity:
                        best_choice_similarity=choice_similarity
                        system_guess_word=choice
        
        #validate label
        if best_choice_similarity==-1:
            label='guess'
            #If label is "guess" then choose the system word randomly
            random_column = str(random.randint(0,3))
            system_guess_word= row[random_column]
        elif answer==system_guess_word:
            label='correct'
        else:
            label='wrong'
        #add each row to the list
        details_list.append([question,answer,system_guess_word,label])
        #print(f"{question},{answer},{system_guess_word},{label}")
    
    return details_list

In [75]:
#generate output file with details
def generate_details_file(model_name,details_list):
    output_file_name=f"{model_name}-details.csv"
    with open(output_file_name, 'w') as output_file:
        for row in details_list:            
            for index in range(0,4):
                if(index==0):                    
                    output_file.write(row[index])  
                else:
                    output_file.write(',' + row[index])
            output_file.write('\n')

In [76]:
#append results from each model to analysis file
def append_results(model_name,vocabulary_size,details_list):
    
    output_file_name = "analysis.csv"
    #C is count_correct
    C = 0
    count_guess = 0
    
    for row in details_list:
        if row[3] == 'correct':            
            C +=1
        elif row[3] == 'guess':
            count_guess +=1
    #V  is number of questions without guess
    V = len(details_list) - count_guess
    model_accuracy = C / V

    with open(output_file_name, 'a') as output_file:
         output_file.write(f"{model_name},{vocabulary_size},{C},{V},{model_accuracy}\n")

In [77]:
# Load dataset
dataframe = pd.read_csv('synonym.csv')

In [3]:
#Task 1

In [5]:
#Load wv_google
wv_google=downloader.load('word2vec-google-news-300')

In [78]:
list_details = get_details(dataframe,wv_google)
generate_details_file('word2vec-google-news-300',list_details)
vocabulary_size = len(wv_google.key_to_index)
append_results('word2vec-google-news-300',vocabulary_size,list_details)